<a href="https://colab.research.google.com/github/ghirailghiro/GPU_Computing_Project/blob/3-first-step-gradient-computation/Gradient_First_Step.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cuda plugin

!pip install nvcc4jupyter
%load_ext nvcc4jupyter
!nvidia-smi

# plugin for cpp sintax highlighting

!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

Source files will be saved in "/tmp/tmpgkadogpx".
/bin/bash: line 1: nvidia-smi: command not found
--2024-02-17 15:59:12--  https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.github.com (gist.github.com)... 140.82.112.4
Connecting to gist.github.com (gist.github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py [following]
--2024-02-17 15:59:12--  https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘cpp_plugin.py’

cpp_

In [2]:
%%cuda
#include <stdio.h>
#include <iostream>

using namespace std;

__global__ void helloFromGPU (void) {
  int tID = threadIdx.x;
  printf("Hello World from GPU (I'am thread %d)!\n", tID);
}

int main(void) {
  //# hello from GPU
  cout << "Hello World from CPU!" << endl;
  cudaSetDevice(1);
  helloFromGPU <<<1, 10>>>();
  cudaDeviceSynchronize();
  return 0;
}

Hello World from CPU!

